In [5]:
import openai
from openai.embeddings_utils import get_embedding, get_embeddings
import os
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [7]:
ENGINE = 'text-embedding-ada-002'

In [9]:
embedded_text = get_embedding('I love to be vectorized', engine=ENGINE)

In [10]:
len(embedded_text)

1536